# RandomForest-SimpeImputer

In [41]:
#importamos las librerias
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import pandas as pd
import os
import numpy as np
from pathlib import Path

In [42]:
#Cargamos la data
root=Path(".").resolve().parent
filename='melb_data.csv'
filepath=os.path.join(root,'data','raw',filename)
df=pd.read_csv(filepath)
print("Dimensiones: ",df.ndim)
print("Shape: ",df.shape)
print(df.head())

Dimensiones:  2
Shape:  (13580, 21)
       Suburb           Address  Rooms Type      Price Method SellerG  \
0  Abbotsford      85 Turner St      2    h  1480000.0      S  Biggin   
1  Abbotsford   25 Bloomburg St      2    h  1035000.0      S  Biggin   
2  Abbotsford      5 Charles St      3    h  1465000.0     SP  Biggin   
3  Abbotsford  40 Federation La      3    h   850000.0     PI  Biggin   
4  Abbotsford       55a Park St      4    h  1600000.0     VB  Nelson   

        Date  Distance  Postcode  ...  Bathroom  Car  Landsize  BuildingArea  \
0  3/12/2016       2.5    3067.0  ...       1.0  1.0     202.0           NaN   
1  4/02/2016       2.5    3067.0  ...       1.0  0.0     156.0          79.0   
2  4/03/2017       2.5    3067.0  ...       2.0  0.0     134.0         150.0   
3  4/03/2017       2.5    3067.0  ...       2.0  1.0      94.0           NaN   
4  4/06/2016       2.5    3067.0  ...       1.0  2.0     120.0         142.0   

   YearBuilt  CouncilArea Lattitude  Longtit

In [43]:
#selccionamos solo las variables numericas
y=df.Price
X=df.copy()
X=X.drop('Price',axis=1)
X=X.select_dtypes(exclude=['object'])
train_X,valid_X,train_y,valid_y=train_test_split(X,y,test_size=0.2,train_size=0.8,random_state=0)

In [44]:
#hacemos una copia de train_X y valid_X
train_X_plus=train_X.copy()
valid_X_plus=valid_X.copy()

#buscamos las columnas con nulls
cols=train_X_plus.columns

cols_nulls=[col for col in train_X.columns if train_X[col].isnull().any()]
cols_nulls



['Car', 'BuildingArea', 'YearBuilt']

In [45]:
#agregamos las columnas con boolean si el registro es null para hacerla seguimiento
for col in cols_nulls:
    train_X_plus[col+"_nulls"]=train_X_plus[col].isnull()
    valid_X_plus[col+"_nulls"]=valid_X_plus[col].isnull()

#aplicamos el imputer
    
imp=SimpleImputer()

train_X_imp=pd.DataFrame(imp.fit_transform(train_X_plus))
valid_X_imp=pd.DataFrame(imp.transform(valid_X_plus))

train_X_imp.columns=train_X_plus.columns
valid_X_imp.columns=valid_X_plus.columns


In [46]:
#habiendo imputado los valores realizamos entrenaremos el modelo

model=RandomForestRegressor(n_estimators=10,random_state=0)
model.fit(train_X_imp,train_y)
predicts=model.predict(valid_X_imp)
mae=mean_absolute_error(valid_y,predicts)
print("MAE: ",mae)



MAE:  178927.503183954
